In [1]:
# Part 3: Find vanishing directions

In [12]:
import numpy as np
import cv2
import sys

In [13]:
fixedMtx = np.array([[982.09741211,   0.        , 609.0936327 ],
       [  0.        , 973.01837158, 455.94049369],
       [  0.        ,   0.        ,   1.        ]])

In [14]:
horizontalLines = np.load('horizontalLines.npy')
verticalLines = np.load('verticalLines.npy')

In [15]:
KInverse = np.linalg.inv(fixedMtx)


In [21]:
def getCameraPoint(pixelPoint):
    return np.dot(KInverse,np.array([pixelPoint[0], pixelPoint[1], 1])) # K-1 * [x y 1] 


# points on pixel frame [[x y] [ x y]]
def getLeverVector(points):
    # n = pc1 x pc2
    pc1 = getCameraPoint(points[0])
    pc2 = getCameraPoint(points[1])
    n = np.cross(pc1, pc2)
    return n
#     return n / np.linalg.norm(n) // don't need to normalize as it doesn't have any effect.


# n1, n2 = lever vectors of two parallel lines.
def getVanishingDirection(n1, n2):
    v = np.cross(n1, n2)
    return v / np.linalg.norm(v)
    

# n is a lever vector and v is vanishing direction computed from two lever vectors != n
def getResidual(n, v):
    r = np.linalg.norm(np.dot(n, v))
    return r


# pre-computed so that we don't need to recompute
def getAllLeverVectors(lines):
    
    leverVectors = []
    for lineIndex in range(lines.shape[0]):
        leverVectors.append(getLeverVector(lines[lineIndex]))
    
    return leverVectors
        
    
def getVPwithMinimalMedianResidualError(lines):
    
    leverVectors = getAllLeverVectors(lines)
    medianResiduals = []
    minResidual = sys.float_info.max
    minI = None
    minJ = None
    minM = None
    minV = None
    
    for m in range(0, 100):
        
        # 1. Get two random lines, their lever vectors, and vanishing direction
        lineIndices = np.random.randint(low = 0, high = lines.shape[0], size = 2).tolist()
        i = lineIndices[0]
        j = lineIndices[1]
        n1 = leverVectors[i]
        n2 = leverVectors[j]
        v = getVanishingDirection(n1, n2)
        
        # compute residual between the vanishining direction and other lever vectors. Ideally it should be close to 0
        residuals = []
        for k in range(lines.shape[0]):
            if k != i and k != j:
                nk = leverVectors[k]
                residuals.append( getResidual(nk, v) )
        
        medResidual = np.median(np.array(residuals))
        medianResiduals.append(medResidual)
        
        if medResidual < minResidual:
            minResidual = medResidual
            minM = m
            minI = i
            minJ = j
            minV = v
            
    
    print(f'minimum residual: {minResidual}')
    return minV
    


In [23]:
horVP = getVPwithMinimalMedianResidualError(horizontalLines)
vertVP = getVPwithMinimalMedianResidualError(verticalLines)    

minimum residual: 0.019776220347348408
minimum residual: 0.0031349455091387057


C:\Users\adhoc\Anaconda3\envs\opencv3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [24]:
np.dot(horVP, vertVP) # should be close to 0.
    

-0.22867823485508093

In [25]:
transversalVP = np.cross(horVP, vertVP)
transversalVP = transversalVP / np.linalg.norm(transversalVP)

In [26]:
transversalVP

array([-0.91223623,  0.0552812 , -0.40591754])